Question 15.2

In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization
problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional
requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem with real data. The data is given in the file diet.xls. 

1.	Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using PuLP.  Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)
2.	Please add to your model the following constraints (which might require adding more variables) and solve the new model:
a.	If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)
b.	Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.
c.	To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered meat?), just call it whatever you think is appropriate – I want you to learn how to write this type of constraint, but I don’t really care whether we agree on how to classify foods!]

If you want to see what a more full-sized problem would look like, try solving your models for the file diet_large.xls, which is a low-cholesterol diet model (rather than minimizing cost, the goal is to minimize cholesterol intake).  I don’t know anyone who’d want to eat this diet – the optimal solution includes dried chrysanthemum garland, raw beluga whale flipper, freeze-dried parsley, etc. – which shows why it’s necessary to add additional constraints beyond the basic ones we saw in the video!
	[Note: there are many optimal solutions, all with zero cholesterol, so you might get a different one.  It probably won’t be much more appetizing than mine.]



Sources Used:
1. https://pypi.org/project/PuLP/
2. https://coin-or.github.io/pulp/
3. BlueJeans Office Hours Recordings

In [1]:
# Load libraries
import pulp
from pulp import *
import pandas as pd
import numpy as np

solver_list = listSolvers()
#Check pulp works properly. Uncomment code to run this
#pulp.pulpTestAll()

In [2]:
#load dataframe
df = pd.read_excel('diet.xls',header=0)
df.tail(10)

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
57,Splt Pea&Hamsoup,0.67,1 C (8 Fl Oz),184.8,7.2,4.0,964.8,26.8,4.1,11.1,4872.0,7.0,33.6,2.1
58,Vegetbeef Soup,0.71,1 C (8 Fl Oz),158.1,10.0,3.8,1915.1,20.4,4.0,11.2,3785.1,4.8,32.6,2.2
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [3]:
#drop the last 3 rows
df.drop(df.tail(3).index,inplace=True)

In [4]:
df.tail(10)

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
54,Pretzels,0.12,1 Oz,108.0,0.0,1.0,486.2,22.5,0.9,2.6,0.0,0.0,10.2,1.2
55,Tortilla Chip,0.19,1 Oz,142.0,0.0,7.4,149.7,17.8,1.8,2.0,55.6,0.0,43.7,0.4
56,Chicknoodl Soup,0.39,1 C (8 Fl Oz),150.1,12.3,4.6,1862.2,18.7,1.5,7.9,1308.7,0.0,27.1,1.5
57,Splt Pea&Hamsoup,0.67,1 C (8 Fl Oz),184.8,7.2,4.0,964.8,26.8,4.1,11.1,4872.0,7.0,33.6,2.1
58,Vegetbeef Soup,0.71,1 C (8 Fl Oz),158.1,10.0,3.8,1915.1,20.4,4.0,11.2,3785.1,4.8,32.6,2.2
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0


In [5]:
#create the optimization problem framework using PuLP method
problem = LpProblem("Diet_Problem", LpMinimize)

In [6]:
#create list of foods
foods = list(df['Foods'])

In [7]:
#create dict of costs
costs = dict(zip(foods,df['Price/ Serving']))

In [8]:
#create dictionaries of other variables
calories = dict(zip(foods,df['Calories']))
cholesterol = dict(zip(foods,df['Cholesterol mg']))
fat = dict(zip(foods,df['Total_Fat g']))
sodium = dict(zip(foods,df['Sodium mg']))
carbohydrates = dict(zip(foods,df['Carbohydrates g']))
fiber = dict(zip(foods,df['Dietary_Fiber g']))
protein = dict(zip(foods,df['Protein g']))
vit_a = dict(zip(foods,df['Vit_A IU']))
vit_c = dict(zip(foods,df['Vit_C IU']))
calcium = dict(zip(foods,df['Calcium mg']))
iron = dict(zip(foods,df['Iron mg']))

In [28]:
calories

{'Frozen Broccoli': 73.8,
 'Carrots,Raw': 23.7,
 'Celery, Raw': 6.4,
 'Frozen Corn': 72.2,
 'Lettuce,Iceberg,Raw': 2.6,
 'Peppers, Sweet, Raw': 20.0,
 'Potatoes, Baked': 171.5,
 'Tofu': 88.2,
 'Roasted Chicken': 277.4,
 'Spaghetti W/ Sauce': 358.2,
 'Tomato,Red,Ripe,Raw': 25.8,
 'Apple,Raw,W/Skin': 81.4,
 'Banana': 104.9,
 'Grapes': 15.1,
 'Kiwifruit,Raw,Fresh': 46.4,
 'Oranges': 61.6,
 'Bagels': 78.0,
 'Wheat Bread': 65.0,
 'White Bread': 65.0,
 'Oatmeal Cookies': 81.0,
 'Apple Pie': 67.2,
 'Chocolate Chip Cookies': 78.1,
 'Butter,Regular': 35.8,
 'Cheddar Cheese': 112.7,
 '3.3% Fat,Whole Milk': 149.9,
 '2% Lowfat Milk': 121.2,
 'Skim Milk': 85.5,
 'Poached Eggs': 74.5,
 'Scrambled Eggs': 99.6,
 'Bologna,Turkey': 56.4,
 'Frankfurter, Beef': 141.8,
 'Ham,Sliced,Extralean': 37.1,
 'Kielbasa,Prk': 80.6,
 "Cap'N Crunch": 119.6,
 'Cheerios': 111.0,
 "Corn Flks, Kellogg'S": 110.5,
 "Raisin Brn, Kellg'S": 115.1,
 'Rice Krispies': 112.2,
 'Special K': 110.8,
 'Oatmeal': 145.1,
 'Malt-O-Meal,C

In [9]:
#create our variables with a 0 lower bound and continuous

#all food items must be at minimum 0 cannot go negative and are continuous variables
amount_var = LpVariable.dicts("Food",foods,lowBound = 0, cat='Continuous')
print(amount_var)

{'Frozen Broccoli': Food_Frozen_Broccoli, 'Carrots,Raw': Food_Carrots,Raw, 'Celery, Raw': Food_Celery,_Raw, 'Frozen Corn': Food_Frozen_Corn, 'Lettuce,Iceberg,Raw': Food_Lettuce,Iceberg,Raw, 'Peppers, Sweet, Raw': Food_Peppers,_Sweet,_Raw, 'Potatoes, Baked': Food_Potatoes,_Baked, 'Tofu': Food_Tofu, 'Roasted Chicken': Food_Roasted_Chicken, 'Spaghetti W/ Sauce': Food_Spaghetti_W__Sauce, 'Tomato,Red,Ripe,Raw': Food_Tomato,Red,Ripe,Raw, 'Apple,Raw,W/Skin': Food_Apple,Raw,W_Skin, 'Banana': Food_Banana, 'Grapes': Food_Grapes, 'Kiwifruit,Raw,Fresh': Food_Kiwifruit,Raw,Fresh, 'Oranges': Food_Oranges, 'Bagels': Food_Bagels, 'Wheat Bread': Food_Wheat_Bread, 'White Bread': Food_White_Bread, 'Oatmeal Cookies': Food_Oatmeal_Cookies, 'Apple Pie': Food_Apple_Pie, 'Chocolate Chip Cookies': Food_Chocolate_Chip_Cookies, 'Butter,Regular': Food_Butter,Regular, 'Cheddar Cheese': Food_Cheddar_Cheese, '3.3% Fat,Whole Milk': Food_3.3%_Fat,Whole_Milk, '2% Lowfat Milk': Food_2%_Lowfat_Milk, 'Skim Milk': Food_Ski

In [10]:
#define objective function
problem += lpSum([costs[i]*amount_var[i] for i in foods])

In [11]:

#calories
problem += lpSum([calories[a] * amount_var[a] for a in foods]) >= 1500, "Minimum Calories"
problem += lpSum([calories[a] * amount_var[a] for a in foods]) <= 2500, "Maximum Calories"

#cholesterol
problem += lpSum([cholesterol[a] * amount_var[a] for a in foods]) >= 30, "Minimum cholesterol"
problem += lpSum([cholesterol[a] * amount_var[a] for a in foods]) <= 240, "Maximum cholesterol"

#fat
problem += lpSum([fat[a] * amount_var[a] for a in foods]) >= 20, "Minimum fat"
problem += lpSum([fat[a] * amount_var[a] for a in foods]) <= 70, "Maximum fat"

#sodium
problem += lpSum([sodium[a] * amount_var[a] for a in foods]) >= 800, "Minimum sodium"
problem += lpSum([sodium[a] * amount_var[a] for a in foods]) <= 2000, "Maximum sodium"

#Carbohydrates
problem += lpSum([carbohydrates[a] * amount_var[a] for a in foods]) >= 130, "Minimum carbohydrates"
problem += lpSum([carbohydrates[a] * amount_var[a] for a in foods]) <= 450, "Maximum carbohydrates"

#Dietary Fiber
problem += lpSum([fiber[a] * amount_var[a] for a in foods]) >= 125, "Minimum fiber"
problem += lpSum([fiber[a] * amount_var[a] for a in foods]) <= 250, "Maximum fiber"

#Protein
problem += lpSum([protein[a] * amount_var[a] for a in foods]) >= 60, "Minimum protein"
problem += lpSum([protein[a] * amount_var[a] for a in foods]) <= 100, "Maximum protein"

#Vit_A
problem += lpSum([vit_a[a] * amount_var[a] for a in foods]) >= 1000, "Minimum vit_a"
problem += lpSum([vit_a[a] * amount_var[a] for a in foods]) <= 10000, "Maximum vit_a"

#VIt_C
problem += lpSum([vit_c[a] * amount_var[a] for a in foods]) >= 400, "Minimum vit_c"
problem += lpSum([vit_c[a] * amount_var[a] for a in foods]) <= 5000, "Maximum vit_c"

#Calcium
problem += lpSum([calcium[a] * amount_var[a] for a in foods]) >= 700, "Minimum calcium"
problem += lpSum([calcium[a] * amount_var[a] for a in foods]) <= 1500, "Maximum calcium"

#Iron
problem += lpSum([iron[a] * amount_var[a] for a in foods]) >= 10, "Minimum iron"
problem += lpSum([iron[a] * amount_var[a] for a in foods]) <= 40, "Maximum iron"

In [12]:
problem.solve()

1

In [13]:
varsDictionary = {}
for v in problem.variables():
    varsDictionary[v.name] = v.varValue
    
print(varsDictionary)

{'Food_2%_Lowfat_Milk': 0.0, 'Food_3.3%_Fat,Whole_Milk': 0.0, 'Food_Apple,Raw,W_Skin': 0.0, 'Food_Apple_Pie': 0.0, 'Food_Bagels': 0.0, 'Food_Banana': 0.0, 'Food_Beanbacn_Soup,W_Watr': 0.0, 'Food_Bologna,Turkey': 0.0, 'Food_Butter,Regular': 0.0, "Food_Cap'N_Crunch": 0.0, 'Food_Carrots,Raw': 0.0, 'Food_Celery,_Raw': 52.64371, 'Food_Cheddar_Cheese': 0.0, 'Food_Cheerios': 0.0, 'Food_Chicknoodl_Soup': 0.0, 'Food_Chocolate_Chip_Cookies': 0.0, "Food_Corn_Flks,_Kellogg'S": 0.0, 'Food_Couscous': 0.0, 'Food_Crm_Mshrm_Soup,W_Mlk': 0.0, 'Food_Frankfurter,_Beef': 0.0, 'Food_Frozen_Broccoli': 0.25960653, 'Food_Frozen_Corn': 0.0, 'Food_Grapes': 0.0, 'Food_Ham,Sliced,Extralean': 0.0, 'Food_Hamburger_W_Toppings': 0.0, 'Food_Hotdog,_Plain': 0.0, 'Food_Kielbasa,Prk': 0.0, 'Food_Kiwifruit,Raw,Fresh': 0.0, 'Food_Lettuce,Iceberg,Raw': 63.988506, 'Food_Macaroni,Ckd': 0.0, 'Food_Malt_O_Meal,Choc': 0.0, 'Food_New_E_Clamchwd,W_Mlk': 0.0, 'Food_Neweng_Clamchwd': 0.0, 'Food_Oatmeal': 0.0, 'Food_Oatmeal_Cookies': 

In [14]:
print("Status:", LpStatus[problem.status])

Status: Optimal


In [15]:
varsDictionary = {}
for v in problem.variables():
    if v.varValue > 0:
        varsDictionary[v.name] = v.varValue
    
print(varsDictionary)

{'Food_Celery,_Raw': 52.64371, 'Food_Frozen_Broccoli': 0.25960653, 'Food_Lettuce,Iceberg,Raw': 63.988506, 'Food_Oranges': 2.2929389, 'Food_Poached_Eggs': 0.14184397, 'Food_Popcorn,Air_Popped': 13.869322}


As outlined in the problem statement, Celery, Broccoli, Iceberg Lettuce, Oranges, Poached Egges, and Popcorn comprise our optimal diet.

For part 2, of this problem. I need to add a few additional constraints and a new variable. Mainly
1) If food is selected, a minimum of 1/10 of a service needs to be added
2) Cannot have celery and broccoli together
3) At least 3 kinds of meat must be chosen

I'm going to re-use my code from above and just add extra code throughout. The objective function is the same and the same constraints are used, only additional ones are added

### Part 2

In [16]:
#add list of protein
protein = ['Roasted Chicken', 'Poached Eggs', 'Scrambled Eggs', 'Bologna,Turkey',
           'Frankfurter, Beef', 'Ham,Sliced,Extralean','Kielbasa,Prk','Pizza W/Pepperoni'
           ,'Hamburger W/Toppings','Hotdog, Plain','Pork','Sardines in Oil','White Tuna in Water',
          'Splt Pea&Hamsoup','Neweng Clamchwd','New E Clamchwd,W/Mlk','Beanbacn Soup,W/Watr'  ]

In [17]:
#add new variable

#This is a binary variable to indicate whether a food was choosen or not. Choosen foods need to be 1/10 of a serving at minimum.
chosen_var = LpVariable.dicts("Chosen",foods,lowBound = 0, upBound = 1, cat='Integer')
print(chosen_var)


{'Frozen Broccoli': Chosen_Frozen_Broccoli, 'Carrots,Raw': Chosen_Carrots,Raw, 'Celery, Raw': Chosen_Celery,_Raw, 'Frozen Corn': Chosen_Frozen_Corn, 'Lettuce,Iceberg,Raw': Chosen_Lettuce,Iceberg,Raw, 'Peppers, Sweet, Raw': Chosen_Peppers,_Sweet,_Raw, 'Potatoes, Baked': Chosen_Potatoes,_Baked, 'Tofu': Chosen_Tofu, 'Roasted Chicken': Chosen_Roasted_Chicken, 'Spaghetti W/ Sauce': Chosen_Spaghetti_W__Sauce, 'Tomato,Red,Ripe,Raw': Chosen_Tomato,Red,Ripe,Raw, 'Apple,Raw,W/Skin': Chosen_Apple,Raw,W_Skin, 'Banana': Chosen_Banana, 'Grapes': Chosen_Grapes, 'Kiwifruit,Raw,Fresh': Chosen_Kiwifruit,Raw,Fresh, 'Oranges': Chosen_Oranges, 'Bagels': Chosen_Bagels, 'Wheat Bread': Chosen_Wheat_Bread, 'White Bread': Chosen_White_Bread, 'Oatmeal Cookies': Chosen_Oatmeal_Cookies, 'Apple Pie': Chosen_Apple_Pie, 'Chocolate Chip Cookies': Chosen_Chocolate_Chip_Cookies, 'Butter,Regular': Chosen_Butter,Regular, 'Cheddar Cheese': Chosen_Cheddar_Cheese, '3.3% Fat,Whole Milk': Chosen_3.3%_Fat,Whole_Milk, '2% Lowfat

Since each individual food needs to be assigned a constraint we need to add atleast 2 constraints for each.

In [18]:
#Add new constraints

#constraint for minimum serving. As discussed in the Office Hours we will need two constraints.
for i in foods:
    problem += amount_var[i] >= chosen_var[i]*0.1
    problem += amount_var[i] <= chosen_var[i]*10000  #note - 10,000 is an arbitary number to indicate inf
    
#constraint for brocolli and celery. If they are chosen, their chosen_var is 1. adding both should be 1 or less.
problem += chosen_var['Frozen Broccoli'] + chosen_var['Celery, Raw'] <= 1

#constraints for 3 types of protein. List comphrension to add chosen_vars. Should be greater than or equal to 3
problem += lpSum([chosen_var[i] for i in protein]) >= 3

In [22]:
problem.solve()
print("Status:", LpStatus[problem.status])

Status: Optimal


In [23]:
varsDictionary = {}
for v in problem.variables():
    if v.varValue > 0:
        varsDictionary[v.name] = v.varValue
    
print(varsDictionary)

{'Chosen_Celery,_Raw': 1.0, 'Chosen_Kielbasa,Prk': 1.0, 'Chosen_Lettuce,Iceberg,Raw': 1.0, 'Chosen_Oranges': 1.0, 'Chosen_Peanut_Butter': 1.0, 'Chosen_Poached_Eggs': 1.0, 'Chosen_Popcorn,Air_Popped': 1.0, 'Chosen_Scrambled_Eggs': 1.0, 'Food_Celery,_Raw': 42.399358, 'Food_Kielbasa,Prk': 0.1, 'Food_Lettuce,Iceberg,Raw': 82.802586, 'Food_Oranges': 3.0771841, 'Food_Peanut_Butter': 1.9429716, 'Food_Poached_Eggs': 0.1, 'Food_Popcorn,Air_Popped': 13.223294, 'Food_Scrambled_Eggs': 0.1}


This is an interesting diet for sure, which makes me want to add more constraints and see how well it does. For one, I think poached eggs and scrambled eggs kind of defeats the purpose of 3 separate proteins. We can add a constraint that is is similar to the Celery/Broccoli one. If you choose one of the eggs, you can't choose the other. Then let's see what we get.

In [26]:
problem += chosen_var['Poached Eggs'] + chosen_var['Scrambled Eggs'] <= 1

In [27]:
problem.solve()
print("Status:", LpStatus[problem.status])
varsDictionary = {}
for v in problem.variables():
    if v.varValue > 0:
        varsDictionary[v.name] = v.varValue
    
print(varsDictionary)

Status: Optimal
{'Chosen_Bologna,Turkey': 1.0, 'Chosen_Celery,_Raw': 1.0, 'Chosen_Kielbasa,Prk': 1.0, 'Chosen_Lettuce,Iceberg,Raw': 1.0, 'Chosen_Oranges': 1.0, 'Chosen_Peanut_Butter': 1.0, 'Chosen_Poached_Eggs': 1.0, 'Chosen_Popcorn,Air_Popped': 1.0, 'Food_Bologna,Turkey': 0.1, 'Food_Celery,_Raw': 42.003189, 'Food_Kielbasa,Prk': 0.1, 'Food_Lettuce,Iceberg,Raw': 83.411008, 'Food_Oranges': 3.0862592, 'Food_Peanut_Butter': 1.9541879, 'Food_Poached_Eggs': 0.12033097, 'Food_Popcorn,Air_Popped': 13.233318}


As expected the scrambled eggs have been removed from the optimal solution and turkey has been added.